In [1]:
import numpy as np
import os
import h5py
import skvideo.io

from pathlib import Path
from typing import List


In [2]:
def find_all_series(data_folders: List[str]) -> List[str]:
    """Find all .h5 series from a list of folders.
    
    Args:
        data_folders: List of paths to folders containing .h5 series.
    
    Returns:
        A list of filenames to .h5 series.
    """
    h5_series = []
    for data_folder in data_folders:
        # h5_series.extend([Path(p).as_posix() for p in glob(f"{data_folder}/*.h5")])
        h5_series.extend([p.as_posix() for p in Path(data_folder).rglob("*.h5")])
    return h5_series


def render_plant_from_h5(h5_path, save_path, decimate=4, slowdown_factor=1, x0=512, x1=1536, overwrite=False):
    """Create an MP4 video from a dataset within an H5 file.
    
    Args:
        h5_path: Path to the H5 file containing the 'vol' dataset with images.
        save_path: Path where the output MP4 video will be saved.
        decimate: Factor by which the images will be decimated (reduced in resolution).
        slowdown_factor: Factor by which the video playback will be slowed down.
        x0, x1: Horizontal bounds for cropping the image.
        overwrite: If False, existing videos will not be overwritten.
    
    Returns:
        None. The video is saved to the specified path if it does not exist or overwrite is True.
    """
    save_path = Path(save_path)
    if save_path.exists() and not overwrite:
        print(f"Video {save_path} already exists. Skipping due to overwrite=False.")
        return
    
    print(f"Opening H5 file: {h5_path}")
    with h5py.File(h5_path, 'r') as f:
        print("Reading 'vol' dataset from H5 file...")
        data = f['vol'][:]
        print(f"Dataset shape: {data.shape}")
    
    out_video = []
    print("Processing images...")
    for i, img in enumerate(data):
        # Crop and decimate
        # img = img[:, x0:x1]
        img = img[::decimate, ::decimate]
        if i % 10 == 0:  # Print progress every 10 frames
            print(f"Processed {i+1}/{len(data)} frames...")
        out_video.append(img)
    
    out_video = np.stack(out_video, axis=0)
    print(f"Total frames before slowdown: {len(out_video)}")
    
    if slowdown_factor > 1:
        out_video = np.repeat(out_video, slowdown_factor, axis=0)
        print(f"Applied slowdown factor. Total frames after slowdown: {len(out_video)}")
    
    # Ensure the video is in the correct format for skvideo to write
    out_video = out_video.astype(np.uint8)
    
    print(f"Writing video to {save_path}...")
    skvideo.io.vwrite(str(save_path), out_video)
    print("Video creation completed.")

In [3]:
# Path to find h5s and save mp4s (1 per h5)
base_dir = "H:/users/eberrigan/20250225_Elohim_Bello_Exp02_timelapse_images/Rice_time_lapse_videos_20250225/h5s_preds_by_frame"

In [4]:
# If overwrite=True mp4s will be overwritten
overwrite = True
decimation = 2 # Factor used to coarsen the video 
# x0, x1 = (550, 1630) # cropping
# x1 = ((x1 - x0) // (4 * decimation)) * (4 * decimation) + x0 # adjusted cropping
slowdown_factor = 1  # must be integer >= 1

In [5]:
# Make Path object
base_dir = Path(base_dir)

# Check current working directory
print(f"Current working directory: {os.getcwd()}")
print(f"Looking in: {base_dir.absolute()}")  # Print the absolute path for clarity

# Ensure the directory exists
if not base_dir.exists():
    print(f"The directory {base_dir.absolute()} does not exist.")
else:
    # Attempt to find and process .h5 files
    # Find all .h5 files within the base directory and its subdirectories
    for h5_file in base_dir.glob('**/*.h5'):
        mp4_file = h5_file.with_suffix('.mp4')
        print(f"Processing {h5_file}...")
        render_plant_from_h5(h5_file, mp4_file, decimate=4, slowdown_factor=slowdown_factor, x0=512, x1=1536, overwrite=overwrite)
        print(f"Made {mp4_file}...")

Current working directory: c:\repos\cylinder-time-lapse
Looking in: H:\users\eberrigan\20250225_Elohim_Bello_Exp02_timelapse_images\Rice_time_lapse_videos_20250225\h5s_preds_by_frame
Processing H:\users\eberrigan\20250225_Elohim_Bello_Exp02_timelapse_images\Rice_time_lapse_videos_20250225\h5s_preds_by_frame\g1_r1.h5...
Opening H5 file: H:\users\eberrigan\20250225_Elohim_Bello_Exp02_timelapse_images\Rice_time_lapse_videos_20250225\h5s_preds_by_frame\g1_r1.h5
Reading 'vol' dataset from H5 file...
Dataset shape: (1008, 1080, 2048, 1)
Processing images...
Processed 1/1008 frames...
Processed 11/1008 frames...
Processed 21/1008 frames...
Processed 31/1008 frames...
Processed 41/1008 frames...
Processed 51/1008 frames...
Processed 61/1008 frames...
Processed 71/1008 frames...
Processed 81/1008 frames...
Processed 91/1008 frames...
Processed 101/1008 frames...
Processed 111/1008 frames...
Processed 121/1008 frames...
Processed 131/1008 frames...
Processed 141/1008 frames...
Processed 151/1008